# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [2]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-0oiphnio
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /tmp/pip-req-build-0oiphnio
  Resolved https://github.com/huggingface/accelerate.git to commit 61bcdaa45d35a77478decc78ce204b54dc108812
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-1.12.0.dev0-py3-none-any.whl size=375796 sha256=c4103b9c1c7414d31910ca101daf6e48baf6824e44924f2eef6a1f15c662b6de
  Stored in directory: /tmp/pip-ephem-wheel-cache-4z1q_682/wheels/5a/20/fb/1221fe933b56fe7ac69fd00159d9a1950bc8ced38198abc18f
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [4]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [5]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [6]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [8]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [14]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [13]:
sp_nl2sql = sp_nl2sql.format(question="What are the names of the customers who placed orders in 2024?")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE 3+ TABLES HERE

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `YOUR QUERY HERE`:
    ```sql3
    


In [12]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [ ]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [22]:
# --- 1. Define the full NL2SQL Template ---
# This is the outer structure with placeholders
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure**


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   {schema}

    ### Samples
    {samples}

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

# --- 2. Define the necessary Database Schema ---
TABLES_SCHEMA = """
CREATE TABLE employees (
  ID_Usr INTEGER PRIMARY KEY,
  name TEXT,
  title TEXT
);
CREATE TABLE salary (
  ID_usr INTEGER,
  salary INTEGER,
  FOREIGN KEY (ID_usr) REFERENCES employees(ID_Usr)
);
CREATE TABLE studies (
  ID_study INTEGER PRIMARY KEY,
  ID_Usr INTEGER,
  degree TEXT,
  FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
);
"""

# --- 3. Define the Sample Queries and Responses (BUG-PROOF METHOD) ---
# Defining content via list joining to avoid the "incomplete input" SyntaxError bug
SAMPLE_RESPONSES_LIST = [
    'Question: "How many employees have the title \'Analyst\'?"',
    '```sql3',
    "SELECT COUNT(ID_Usr) FROM employees WHERE title = 'Analyst';",
    '```',
    '\n',
    'Question: "What is the average salary of all employees?"',
    '```sql3',
    'SELECT AVG(salary) FROM salary;',
    '```'
]
# Join the list into a single string separated by newlines
SAMPLE_RESPONSES = "\n".join(SAMPLE_RESPONSES_LIST)


# --- 4. Define the Question and Format the Final Prompt ---
TEST_QUESTION = "Return the name of the highest-paid employee."

final_nl2sql_prompt = sp_nl2sql2.format(
    schema=TABLES_SCHEMA,
    samples=SAMPLE_RESPONSES,
    question=TEST_QUESTION
)

# --- 5. Print the resulting runnable prompt ---
print(final_nl2sql_prompt)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure**


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   
CREATE TABLE employees (
  ID_Usr INTEGER PRIMARY KEY,
  name TEXT,
  title TEXT
);
CREATE TABLE salary (
  ID_usr INTEGER,
  salary INTEGER,
  FOREIGN KEY (ID_usr) REFERENCES employees(ID_Usr)
);
CREATE TABLE studies (
  ID_study INTEGER PRIMARY KEY,
  ID_Usr INTEGER,
  degree TEXT,
  FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
);


    ### Samples
    Question: "How many employees have the title 'Analyst'?"
```sql3
SELECT COUNT(ID_Usr) FROM employees WHERE title = 'Analyst';
```


Question: 

In [25]:
# Extracts the SQL code from the model's raw text output
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT COUNT(*) AS total_students FROM students WHERE gender = 'female' AND age >= 18 AND age <= 24;


In [ ]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [ ]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [36]:
# --- 1. Define the full NL2SQL Template ---
# This template requires {schema}, {samples}, and {question} keywords.
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure**


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   {schema}

    ### Samples
    {samples}

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

# --- 2. Define the content for the {schema} placeholder ---
TABLES_SCHEMA = """
CREATE TABLE employees (
  ID_Usr INTEGER PRIMARY KEY,
  name TEXT,
  title TEXT
);
CREATE TABLE salary (
  ID_usr INTEGER,
  salary INTEGER,
  FOREIGN KEY (ID_usr) REFERENCES employees(ID_Usr)
);
CREATE TABLE studies (
  ID_study INTEGER PRIMARY KEY,
  ID_Usr INTEGER,
  degree TEXT,
  FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
);
"""

# --- 3. Define the content for the {samples} placeholder (Few-Shot Examples) ---
# Using the bug-proof list joining method to define the content
SAMPLE_RESPONSES_LIST = [
    'Question: "How many employees have the title \'Analyst\'?"',
    '```sql3',
    "SELECT COUNT(ID_Usr) FROM employees WHERE title = 'Analyst';",
    '```',
    '\nQuestion: "What is the average salary of all employees?"',
    '```sql3',
    'SELECT AVG(salary) FROM salary;',
    '```'
]
SAMPLE_RESPONSES = "\n".join(SAMPLE_RESPONSES_LIST)


# --- 4. Define the Question and Execute the Formatting ---
TEST_QUESTION = "Return The name of the best paid employee"

# This is the line from your prompt, now executed correctly because all variables are defined above.
sp_nl2sql3 = sp_nl2sql3b.format(
    schema=TABLES_SCHEMA,    # Fills the {schema} placeholder
    samples=SAMPLE_RESPONSES, # Fills the {samples} placeholder
    question=TEST_QUESTION    # Fills the {question} placeholder
)

# --- 5. Print the final, runnable prompt ---
print(sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure**


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

   
CREATE TABLE employees (
  ID_Usr INTEGER PRIMARY KEY,
  name TEXT,
  title TEXT
);
CREATE TABLE salary (
  ID_usr INTEGER,
  salary INTEGER,
  FOREIGN KEY (ID_usr) REFERENCES employees(ID_Usr)
);
CREATE TABLE studies (
  ID_study INTEGER PRIMARY KEY,
  ID_Usr INTEGER,
  degree TEXT,
  FOREIGN KEY (ID_Usr) REFERENCES employees(ID_Usr)
);


    ### Samples
    Question: "How many employees have the title 'Analyst'?"
```sql3
SELECT COUNT(ID_Usr) FROM employees WHERE title = 'Analyst';
```

Question: "

In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)

In [ ]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [40]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_usr GROUP BY employees.name ORDER BY max_salary DESC LIMIT 1;


#Now the question in spanish.


In [ ]:
sp_nl2sql3 = sp_nl2sql3b.format(question="YOUR QUERY HERE")
print (sp_nl2sql3)

In [39]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [38]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT COUNT(*) AS total_students FROM students WHERE gender = 'female' AND age >= 18 AND age <= 24;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?